<a href="https://colab.research.google.com/github/jfkwage/my_try/blob/main/jarvis-tools-notebooks/alignn_jarvis_leaderboard_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook shows how to add benchmarks to [JARVIS-Leaderboard](https://pages.nist.gov/jarvis_leaderboard/) using [alignn](https://github.com/usnistgov/alignn) for [exfoliation_energy](https://pages.nist.gov/jarvis_leaderboard/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy/) task.

The Atomistic Line Graph Neural Network (https://www.nature.com/articles/s41524-021-00650-1) introduces a new graph convolution layer that explicitly models both two and three body interactions in atomistic systems.

This is achieved by composing two edge-gated graph convolution layers, the first applied to the atomistic line graph L(g) (representing triplet interactions) and the second applied to the atomistic bond graph g (representing pair interactions).

The atomistic graph g consists of a node for each atom i (with atom/node representations hi), and one edge for each atom pair within a cutoff radius (with bond/pair representations eij).

The atomistic line graph L(g) represents relationships between atom triplets: it has nodes corresponding to bonds (sharing representations eij with those in g) and edges corresponding to bond angles (with angle/triplet representations tijk).

The line graph convolution updates the triplet representations and the pair representations; the direct graph convolution further updates the pair representations and the atom representations.

This model is used for developing unified graph neural force-fields as well (https://pubs.rsc.org/en/content/articlehtml/2023/dd/d2dd00096b).

First, let's install alignn

In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:10
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check()

In [2]:
# %%time
# !pip install -q dgl==1.0.1+cu117 -f https://data.dgl.ai/wheels/cu117/repo.html
# !pip install -q alignn
%%time
!conda install alignn dgl=2.1.0 pytorch torchvision torchaudio pytorch-cuda -c pytorch -c nvidia -y --quiet

Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - alignn
    - dgl=2.1.0
    - pytorch
    - pytorch-cuda
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    absl-py-2.1.0              |     pyhd8ed1ab_1         105 KB  conda-forge
    alignn-2024.10.30          |     pyhd8ed1ab_0          76 KB  conda-forge
    annotated-types-0.7.0      |     pyhd8ed1ab_1          18 KB  conda-forge
    ase-3.23.0                 |     pyhd8ed1ab_0         1.8 MB  conda-forge
    astunparse-1.6.3           |     pyhd8ed1ab_3          18 KB  conda-forge
    babel-2.16.0               |     pyhd8ed1ab_1         6.2 MB  conda-forge
    blinker-1.9.0       

Now, let's install jarvis_leaderboard

In [18]:
%%time
pip install -q jarvis_leaderboard

CPU times: user 9.85 ms, sys: 0 ns, total: 9.85 ms
Wall time: 1.51 s


Check if you can import libraries

In [19]:
import jarvis_leaderboard

In [5]:
import alignn

Let's populate a dataset (AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae). Files will be saved in the output_path i.e. 'Out' directory

预训练模型尝试：

In [32]:
!pretrained.py -h

/usr/local/lib/python3.10/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
usage: pretrained.py [-h] [--model_name MODEL_NAME] [--file_format FILE_FORMAT]
                     [--file_path FILE_PATH] [--cutoff CUTOFF] [--max_neighbors MAX_NEIGHBORS]

Atomistic Line Graph Neural Network Pretrained Models

options:
  -h, --help            show this help message and exit
  --model_name MODEL_NAME
                        Choose a model from these 57 models:jv_formation_energy_peratom_alignn,
                        jv_optb88vdw_total_energy_alignn, jv_optb88vdw_bandgap_alignn,
               